# Kapitel 5: Objektzustand, Persistenz & Fehlerbehandlung

**Heutige Lernziele:**

* **Objektkopien:** Wir analysieren den fundamentalen und kritischen Unterschied zwischen einer **flachen Kopie (Shallow Copy)** und einer **tiefen Kopie (Deep Copy)**.
* **Serialisierung:** Wir meistern die Techniken, um Objekte mit `pickle` und `shelve` zu speichern und zu laden.
* **Fortgeschrittene Ausnahmebehandlung:** Wir lernen, wie man professionell mit komplexen Fehlersituationen umgeht, indem wir Exceptions verketten und analysieren.

---
---

## **5.1: Objektkopien: Zwillinge und Klone**

### **Das fundamentale Problem – Variablen sind nur Etiketten**

Bevor wir über das Kopieren sprechen, müssen wir ein Kernkonzept von Python verstehen: Eine Zuweisung mit `=` erstellt **niemals** eine Kopie eines komplexen Objekts (wie einer Liste oder einer Klasseninstanz).

* **Konzept:** Eine Variable ist nur ein **Name** oder ein **Etikett**, das auf ein Objekt im Speicher verweist. Eine Zuweisung wie `objekt_b = objekt_a` erstellt lediglich ein zweites Etikett, das auf **dasselbe** Objekt im Speicher zeigt.
* **Analogie:** Stellen Sie sich vor, eine Person heißt "Dr. Anna Schmidt". Ihr Spitzname ist "Anni". `objekt_a` ist "Dr. Anna Schmidt", `objekt_b` ist "Anni". Egal, welchen Namen Sie verwenden, Sie sprechen immer mit derselben Person. Ändert "Anni" ihre Haarfarbe, hat auch "Dr. Anna Schmidt" eine neue Haarfarbe.

**Der Beweis mit `id()`:** Die eingebaute Funktion `id()` gibt die eindeutige Speicheradresse eines Objekts zurück. Wenn zwei Variablen dieselbe `id()` haben, zeigen sie auf dasselbe Objekt.


In [1]:


# Eine einfache Liste im Speicher
liste_a = [1, 2, 3]

# Wir erstellen ein zweites Etikett für DIESELBE Liste
liste_b = liste_a

print(f"ID von liste_a: {id(liste_a)}")
print(f"ID von liste_b: {id(liste_b)}")
print(f"Sind die IDs identisch? {liste_a is liste_b}")

# Was passiert, wenn wir die Liste über das Etikett 'liste_b' verändern?
print(f"\nÄndere liste_b...")
liste_b.append(4)

print(f"liste_b ist jetzt: {liste_b}")
print(f"liste_a wurde ebenfalls verändert: {liste_a}")



ID von liste_a: 4474510208
ID von liste_b: 4474510208
Sind die IDs identisch? True

Ändere liste_b...
liste_b ist jetzt: [1, 2, 3, 4]
liste_a wurde ebenfalls verändert: [1, 2, 3, 4]



Dieses Verhalten ist der Grund, warum wir explizite Kopiermechanismen benötigen.

---

### **Shallow Copy (`copy.copy()`) – Die trügerische Kopie**

Die erste Lösung für unser Problem ist die "flache Kopie", die wir mit der `copy()`-Funktion aus dem `copy`-Modul erstellen.

* **Was es tut:** Eine flache Kopie erstellt ein **neues Top-Level-Objekt**. Wenn Sie eine Liste kopieren, erhalten Sie eine komplett neue Liste mit einer neuen Speicheradresse.
* **Die Gefahr:** Die Elemente *innerhalb* der neuen Liste sind aber nur **Referenzen** auf die ursprünglichen Elemente. Bei einfachen Datentypen wie Zahlen ist das kein Problem. Aber bei **verschachtelten Objekten** (wie Listen in einer Liste oder Instanzen in einer Liste) zeigen beide Listen auf dieselben verschachtelten Objekte.
* **Analogie:** Sie machen eine **Fotokopie eines Dokuments, auf dem ein originaler Post-it-Zettel klebt**. Sie erhalten ein neues, separates Blatt Papier (der neue Container), aber der Post-it-Zettel (das verschachtelte Objekt) ist immer noch dasselbe Original. Wenn jemand auf dem Post-it-Zettel etwas durchstreicht, ist es auf Ihrer Kopie ebenfalls durchgestrichen.

---

### **Deep Copy (`copy.deepcopy()`) – Der perfekte Klon**

Die vollständige und sichere Lösung ist die "tiefe Kopie" mit der `deepcopy()`-Funktion.

* **Was es tut:** Eine tiefe Kopie erstellt einen **vollständigen, rekursiven Klon** des Objekts und aller seiner verschachtelten Objekte. Nichts wird mehr geteilt.
* **Der Prozess:** `deepcopy` durchläuft die gesamte Objektstruktur. Es erstellt eine neue Kopie des Top-Level-Objekts, dann eine neue Kopie jedes verschachtelten Objekts, dann eine neue Kopie der Objekte in diesen Objekten und so weiter, bis alles dupliziert ist.
* **Analogie:** Dies ist die **Fotokopie des Dokuments *und* des Post-it-Zettels**. Alles, was Sie auf Ihrer Kopie ändern, hat keinerlei Auswirkung auf das Original.

---

### **Beispiel A: Die verschachtelte Liste**

Hier sehen wir alle drei Verhaltensweisen im direkten Vergleich.


In [2]:


import copy

# Unser Original-Objekt mit einer verschachtelten Liste
original_liste = [1, [10, 20]]

# Fall 1: Referenzzuweisung
ref_liste = original_liste
print(f"--- Fall 1: Referenz ---")
print(f"Original und Referenz zeigen auf dasselbe Objekt: {original_liste is ref_liste}")
print(f"Die inneren Listen sind ebenfalls dasselbe Objekt: {original_liste[1] is ref_liste[1]}")

# Fall 2: Shallow Copy
shallow_liste = copy.copy(original_liste)
print(f"\n--- Fall 2: Shallow Copy ---")
print(f"Original und Kopie sind verschiedene Objekte: {original_liste is not shallow_liste}")
print(f"Die inneren Listen sind aber immer noch dasselbe Objekt: {original_liste[1] is shallow_liste[1]}")

# Fall 3: Deep Copy
deep_liste = copy.deepcopy(original_liste)
print(f"\n--- Fall 3: Deep Copy ---")
print(f"Original und Kopie sind verschiedene Objekte: {original_liste is not deep_liste}")
print(f"Die inneren Listen sind nun auch verschiedene Objekte: {original_liste[1] is not deep_liste}")

# --- DER TEST: Wir ändern die innere Liste der Shallow Copy ---
print("\n--- DER TEST ---")
print(f"Original-Liste vorher: {original_liste}")
print("Ändere die innere Liste der *Shallow Copy*...")
shallow_liste[1].append(30)

print(f"Shallow Copy nachher: {shallow_liste}")
print(f"Original-Liste wurde mitverändert (korrumpiert!): {original_liste}")
print(f"Deep Copy bleibt unberührt: {deep_liste}")



--- Fall 1: Referenz ---
Original und Referenz zeigen auf dasselbe Objekt: True
Die inneren Listen sind ebenfalls dasselbe Objekt: True

--- Fall 2: Shallow Copy ---
Original und Kopie sind verschiedene Objekte: True
Die inneren Listen sind aber immer noch dasselbe Objekt: True

--- Fall 3: Deep Copy ---
Original und Kopie sind verschiedene Objekte: True
Die inneren Listen sind nun auch verschiedene Objekte: True

--- DER TEST ---
Original-Liste vorher: [1, [10, 20]]
Ändere die innere Liste der *Shallow Copy*...
Shallow Copy nachher: [1, [10, 20, 30]]
Original-Liste wurde mitverändert (korrumpiert!): [1, [10, 20, 30]]
Deep Copy bleibt unberührt: [1, [10, 20]]



---

### **Beispiel B: Die `Reise`-Klasse**

Dieses Problem tritt genauso bei unseren eigenen Klassen auf.


In [3]:


import copy

class Teilnehmer:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name

class Reise:
    def __init__(self, ziel, teilnehmer_liste):
        self.ziel = ziel
        self.teilnehmer = teilnehmer_liste # Eine Liste von Objekten

# Original-Reise
teilnehmer = [Teilnehmer("Anna"), Teilnehmer("Ben")]
reise_original = Reise("Paris", teilnehmer)
print(f"Original-Reise: Ziel {reise_original.ziel}, Teilnehmer {reise_original.teilnehmer}")

# Erstellen einer Shallow Copy für eine alternative Planung
reise_kopie_shallow = copy.copy(reise_original)
reise_kopie_shallow.ziel = "Lyon" # Das Ziel der Kopie zu ändern ist sicher.

# Jetzt storniert ein Teilnehmer auf der alternativen Reise
print("\nBen storniert seine Teilnahme auf der alternativen Reise...")
reise_kopie_shallow.teilnehmer.pop() # Entfernt Ben

print(f"Alternative Reise: Ziel {reise_kopie_shallow.ziel}, Teilnehmer {reise_kopie_shallow.teilnehmer}")
# FATALER FEHLER: Ben ist auch aus der Original-Reise verschwunden!
print(f"Original-Reise: Ziel {reise_original.ziel}, Teilnehmer {reise_original.teilnehmer}") 

print("\n--- MIT DEEP COPY ---")
# Neuer Versuch mit einer tiefen Kopie
reise_original_2 = Reise("Rom", [Teilnehmer("Carla"), Teilnehmer("David")])
reise_kopie_deep = copy.deepcopy(reise_original_2)

# David storniert auf der tiefen Kopie
reise_kopie_deep.teilnehmer.pop()
print(f"Tiefe Kopie: {reise_kopie_deep.teilnehmer}")
print(f"Original bleibt unberührt: {reise_original_2.teilnehmer}")



Original-Reise: Ziel Paris, Teilnehmer [Anna, Ben]

Ben storniert seine Teilnahme auf der alternativen Reise...
Alternative Reise: Ziel Lyon, Teilnehmer [Anna]
Original-Reise: Ziel Paris, Teilnehmer [Anna]

--- MIT DEEP COPY ---
Tiefe Kopie: [Carla]
Original bleibt unberührt: [Carla, David]



**Merksatz:** Sobald Ihre Objekte andere veränderliche Objekte (Listen, Dictionaries, andere Instanzen) enthalten, ist `deepcopy` fast immer die sicherere Wahl für das Erstellen von unabhängigen Duplikaten.

---

### **5.1.2 Transfer auf das `Raumschiff`-Projekt**

Wir demonstrieren die Gefahr einer flachen Kopie eines `Raumschiffs`, das eine Liste von `WaffenSystem`-Objekten als Attribut hat.


In [4]:


import copy

class WaffenSystem:
    def __init__(self, typ, status="Online"):
        self.typ = typ
        self.status = status
    def __repr__(self):
        return f"{self.typ}({self.status})"

class Raumschiff:
    def __init__(self, name, waffen):
        self.name = name
        self.waffensysteme = waffen
        
    def __str__(self):
        return f"Schiff '{self.name}', Waffensysteme: {self.waffensysteme}"

# --- Das Szenario ---
# Originalschiff erstellen
waffen_enterprise = [WaffenSystem("Phaser"), WaffenSystem("Photonentorpedo")]
enterprise = Raumschiff("Enterprise", waffen_enterprise)
print(f"Originalzustand: {enterprise}")

# Wir erstellen eine Shallow Copy für eine Simulation, in der ein System sabotiert wird
print("\nErstelle Shallow Copy für Simulation...")
enterprise_sim_shallow = copy.copy(enterprise)

# Sabotage im Simulator
print("Sabotiere Phaser im Simulator...")
enterprise_sim_shallow.waffensysteme[0].status = "Offline"

# Ergebnis
print(f"Simulator-Schiff: {enterprise_sim_shallow}")
print(f"ECHTES SCHIFF wurde ebenfalls sabotiert: {enterprise}")

# --- Die Lösung mit Deep Copy ---
print("\n--- NEUER VERSUCH MIT DEEP COPY ---")
# Originalschiff zurücksetzen
waffen_enterprise_2 = [WaffenSystem("Phaser"), WaffenSystem("Photonentorpedo")]
enterprise_2 = Raumschiff("Enterprise", waffen_enterprise_2)
print(f"Originalzustand: {enterprise_2}")

# Tiefe Kopie erstellen
enterprise_sim_deep = copy.deepcopy(enterprise_2)

# Sabotage im Simulator
print("Sabotiere Phaser im sicheren Deep-Copy-Simulator...")
enterprise_sim_deep.waffensysteme[0].status = "Offline"

# Ergebnis
print(f"Simulator-Schiff (Deep Copy): {enterprise_sim_deep}")
print(f"ECHTES SCHIFF bleibt unberührt und einsatzbereit: {enterprise_2}")




Originalzustand: Schiff 'Enterprise', Waffensysteme: [Phaser(Online), Photonentorpedo(Online)]

Erstelle Shallow Copy für Simulation...
Sabotiere Phaser im Simulator...
Simulator-Schiff: Schiff 'Enterprise', Waffensysteme: [Phaser(Offline), Photonentorpedo(Online)]
ECHTES SCHIFF wurde ebenfalls sabotiert: Schiff 'Enterprise', Waffensysteme: [Phaser(Offline), Photonentorpedo(Online)]

--- NEUER VERSUCH MIT DEEP COPY ---
Originalzustand: Schiff 'Enterprise', Waffensysteme: [Phaser(Online), Photonentorpedo(Online)]
Sabotiere Phaser im sicheren Deep-Copy-Simulator...
Simulator-Schiff (Deep Copy): Schiff 'Enterprise', Waffensysteme: [Phaser(Offline), Photonentorpedo(Online)]
ECHTES SCHIFF bleibt unberührt und einsatzbereit: Schiff 'Enterprise', Waffensysteme: [Phaser(Online), Photonentorpedo(Online)]


## **5.2: Serialisierung: Objekte für die Ewigkeit**

Bisher existieren unsere Objekte nur im Arbeitsspeicher (RAM), solange unser Programm läuft. Sobald das Programm endet, sind alle Instanzen und ihre Zustände unwiderruflich verloren.

**Serialisierung** ist der Prozess, ein Objekt aus dem Speicher in ein Format umzuwandeln (typischerweise einen **Byte-Stream**), das auf einem persistenten Medium (wie einer Festplatte) gespeichert oder über ein Netzwerk übertragen werden kann. Der umgekehrte Prozess, aus dem Byte-Stream wieder ein vollwertiges Objekt zu erstellen, nennt sich **Deserialisierung**.

In diesem Kapitel lernen wir die Standard-Werkzeuge von Python kennen, um unsere Objekte "haltbar zu machen".

---
---

### **5.2.1 Theorie & Beispiele: `pickle` – Das Einmachglas für Objekte**

**Tiefe Erklärung: Was ist `pickle` und warum brauchen wir es?**

Das `pickle`-Modul ist Pythons eingebauter Mechanismus zur Serialisierung. Es kann fast jedes Python-Objekt – einschließlich Instanzen unserer eigenen Klassen – in eine Folge von Bytes "einlegen" und es später exakt wiederherstellen.

**Anwendungsfälle:**
* **Zustand speichern:** Der klassische Anwendungsfall ist das Speichern des Fortschritts in einem Spiel oder einer Anwendung. Wir nehmen das `Spielstand`-Objekt mit all seinen Attributen und "picklen" es in eine Speicherdatei.
* **Datenübertragung:** Um komplexe Python-Objekte zwischen verschiedenen Prozessen oder über ein Netzwerk an ein anderes Python-Programm zu senden.
* **Caching:** Ergebnisse von aufwändigen Berechnungen können als Pickle-Datei zwischengespeichert werden, um sie beim nächsten Mal schnell wieder laden zu können, anstatt sie neu zu berechnen.

**Die Kernfunktionen:**
* **`pickle.dump(obj, file)`:** Nimmt ein Python-Objekt `obj` und ein Datei-Objekt `file`, das im **binären Schreibmodus (`'wb'`)** geöffnet sein muss. Es schreibt den Byte-Stream des Objekts in die Datei.
* **`pickle.load(file)`:** Nimmt ein Datei-Objekt `file`, das im **binären Lesemodus (`'rb'`)** geöffnet sein muss. Es liest den Byte-Stream und rekonstruiert das ursprüngliche Python-Objekt im Speicher mit all seinen Attributen und Methoden.
* **`dumps()` und `loads()`:** Manchmal wollen wir das Objekt nicht in eine Datei schreiben, sondern den Byte-Stream direkt in einer Variable haben (z.B. um ihn über das Netzwerk zu senden). Dafür gibt es:
    * `pickle.dumps(obj)` (dump string): Gibt den Byte-Stream als `bytes`-Objekt zurück.
    * `pickle.loads(byte_string)` (load string): Rekonstruiert ein Objekt aus einem `bytes`-Objekt.

> **⚠️ Wichtige Sicherheitswarnung:**
> Das `pickle`-Format ist nicht sicher gegen fehlerhafte oder böswillig erstellte Daten. Laden Sie **niemals** eine Pickle-Datei aus einer Quelle, der Sie nicht zu 100% vertrauen. Das Ausführen von `pickle.load()` auf einer manipulierten Datei kann **beliebigen schadhaften Code** auf Ihrem Computer ausführen. Für den Datenaustausch mit nicht vertrauenswürdigen Quellen sind Formate wie JSON oder XML immer vorzuziehen.

---

### **Beispiel A: Ein komplexes Dictionary speichern**


In [5]:


import pickle

# Unser Original-Objekt
daten = {
    "user_id": 123,
    "user_name": "Anna",
    "berechtigungen": ["lesen", "schreiben"],
    "letzter_login": (2025, 7, 4)
}

# --- SPEICHERN ---
# Die Datei muss im 'wb' (write binary) Modus geöffnet werden
with open("meine_daten.pkl", "wb") as datei:
    print("Speichere Datenobjekt in 'meine_daten.pkl'...")
    pickle.dump(daten, datei)
    
print("Speichern abgeschlossen.")

# --- LADEN ---
# Die Datei muss im 'rb' (read binary) Modus geöffnet werden
with open("meine_daten.pkl", "rb") as datei:
    print("\nLade Datenobjekt aus 'meine_daten.pkl'...")
    geladene_daten = pickle.load(datei)
    
print("Laden abgeschlossen.")
print(f"Geladenes Objekt: {geladene_daten}")
print(f"Typ des geladenen Objekts: {type(geladene_daten)}")
print(f"Inhalt ist identisch? {daten == geladene_daten}")



Speichere Datenobjekt in 'meine_daten.pkl'...
Speichern abgeschlossen.

Lade Datenobjekt aus 'meine_daten.pkl'...
Laden abgeschlossen.
Geladenes Objekt: {'user_id': 123, 'user_name': 'Anna', 'berechtigungen': ['lesen', 'schreiben'], 'letzter_login': (2025, 7, 4)}
Typ des geladenen Objekts: <class 'dict'>
Inhalt ist identisch? True



---

### **Beispiel B: Einen `Spielstand` speichern und laden**

Hier zeigen wir die wahre Stärke von `pickle`: Das Speichern einer kompletten Klasseninstanz.


In [6]:


import pickle

class Spielstand:
    def __init__(self, spielername, level, inventar, position):
        self.spielername = spielername
        self.level = level
        self.inventar = inventar # Eine Liste
        self.position = position # Ein Tupel (x, y)
        
    def level_up(self):
        self.level += 1
        print(f"{self.spielername} ist jetzt Level {self.level}!")
        
    def __repr__(self):
        return f"Spielstand(Spieler: {self.spielername}, Lvl: {self.level}, Inventar: {self.inventar})"

# --- Szenario 1: Spiel wird gespielt und gespeichert ---
print("--- SPIELSITZUNG 1 ---")
# Erstellen einer Instanz
aktueller_spielstand = Spielstand("Aragon", 5, ["Schwert", "Heiltrank"], (102, 345))
aktueller_spielstand.level_up()
print(f"Aktueller Zustand: {aktueller_spielstand}")

# Speichern des gesamten Objekts
with open("savegame.dat", "wb") as save_file:
    pickle.dump(aktueller_spielstand, save_file)
print("Spielstand wurde gespeichert. Programm wird beendet.")

# --- Szenario 2: Programm wird neu gestartet und Spielstand geladen ---
print("\n--- SPIELSITZUNG 2 ---")
# Das 'aktueller_spielstand'-Objekt existiert nicht mehr. Wir laden es neu.
with open("savegame.dat", "rb") as load_file:
    geladener_spielstand = pickle.load(load_file)

print("Spielstand wurde geladen!")
print(f"Geladener Zustand: {geladener_spielstand}")

# Das geladene Objekt ist eine vollwertige Instanz mit all ihren Methoden!
geladener_spielstand.level_up()
print(f"Neuer Zustand: {geladener_spielstand}")



--- SPIELSITZUNG 1 ---
Aragon ist jetzt Level 6!
Aktueller Zustand: Spielstand(Spieler: Aragon, Lvl: 6, Inventar: ['Schwert', 'Heiltrank'])
Spielstand wurde gespeichert. Programm wird beendet.

--- SPIELSITZUNG 2 ---
Spielstand wurde geladen!
Geladener Zustand: Spielstand(Spieler: Aragon, Lvl: 6, Inventar: ['Schwert', 'Heiltrank'])
Aragon ist jetzt Level 7!
Neuer Zustand: Spielstand(Spieler: Aragon, Lvl: 7, Inventar: ['Schwert', 'Heiltrank'])



---
---

### **5.2.2 `shelve` – Das persistente Dictionary**

**Erklärung: `pickle` für Faule?**

Stellen Sie sich vor, Sie wollen nicht nur ein Objekt, sondern viele Objekte in einer Datei speichern und sie einzeln über einen Schlüssel (wie in einem Dictionary) abrufen. Mit `pickle` wäre das umständlich: Sie müssten ein Dictionary erstellen, alle Objekte dort hineinlegen, das gesamte Dictionary picklen, es später komplett laden, das gewünschte Objekt heraussuchen und am Ende wieder das gesamte, geänderte Dictionary speichern.

Hierfür bietet Python eine viel bequemere Lösung: das `shelve`-Modul.

* **Konzept:** Ein "Shelf" (Regal) verhält sich für Sie wie ein **persistentes Dictionary**. Es ist ein Objekt, das auf eine Datei auf der Festplatte zugreift.
* **Mechanismus:** Im Hintergrund verwendet `shelve` ein Datenbanksystem und `pickle`, um die Objekte zu speichern und zu laden. Dieser komplexe Prozess wird aber vollständig vor Ihnen verborgen.
* **Workflow:**
    1.  `shelve.open('dateiname')` öffnet die Datenbankdatei (oder erstellt sie).
    2.  Sie arbeiten mit dem zurückgegebenen Objekt wie mit einem normalen Dictionary: `shelf['key'] = mein_objekt`, `x = shelf['key']`, `del shelf['key']`.
    3.  `shelf.close()` schließt die Datei und stellt sicher, dass alle Änderungen geschrieben wurden. **Wichtig:** Die Verwendung eines `with`-Blocks ist die beste Methode, da das Schließen dann automatisch geschieht.

---

### **Beispiel: Ein `MitarbeiterArchiv` mit `shelve`**


In [7]:


import shelve

class Mitarbeiter:
    def __init__(self, name, abteilung):
        self.name = name
        self.abteilung = abteilung
    def __repr__(self):
        return f"Mitarbeiter({self.name}, {self.abteilung})"

# Mitarbeiter erstellen
m1 = Mitarbeiter("Anna", "IT")
m2 = Mitarbeiter("Ben", "HR")
m3 = Mitarbeiter("Carla", "Marketing")

# --- Daten speichern ---
# 'with' sorgt dafür, dass die Datei am Ende sicher geschlossen wird.
with shelve.open("mitarbeiter_db") as archiv:
    print("Speichere Mitarbeiter im Archiv...")
    archiv["101"] = m1 # Personalnummer als Schlüssel
    archiv["102"] = m2
    archiv["103"] = m3
    print("Archiv wurde aktualisiert.")

# --- Daten abrufen und bearbeiten in einer neuen "Sitzung" ---
print("\n--- NÄCHSTER TAG ---")
with shelve.open("mitarbeiter_db") as archiv:
    print(f"Gesamtzahl der Einträge im Archiv: {len(archiv)}")
    
    # Einen Mitarbeiter abrufen
    ben = archiv["102"]
    print(f"Abgerufen: {ben}")
    
    # Einen Mitarbeiter löschen
    print("Lösche Carla (103)...")
    if "103" in archiv:
        del archiv["103"]
    
    # Einen Mitarbeiter aktualisieren
    anna_updated = archiv["101"]
    anna_updated.abteilung = "IT-Leitung"
    archiv["101"] = anna_updated # Wichtig: Objekt muss erneut zugewiesen werden!
    
    print("\nArchiv nach Änderungen:")
    for key, value in archiv.items():
        print(f"- {key}: {value}")



Speichere Mitarbeiter im Archiv...
Archiv wurde aktualisiert.

--- NÄCHSTER TAG ---
Gesamtzahl der Einträge im Archiv: 3
Abgerufen: Mitarbeiter(Ben, HR)
Lösche Carla (103)...

Archiv nach Änderungen:
- 101: Mitarbeiter(Anna, IT-Leitung)
- 102: Mitarbeiter(Ben, HR)



---
---

### **5.2.3 Transfer auf das `Raumschiff`-Projekt**

Wir wenden nun beide Techniken auf unser Projekt an.

**Anwendung von `pickle`:** Wir speichern einen spezifischen, vollständig konfigurierten Schiffsbauplan in eine einzelne Datei.


In [8]:


import pickle

# Nehmen wir an, wir haben eine komplexere Raumschiff-Klasse
class Schiffsklasse:
    def __init__(self, name, klasse, waffen_liste):
        self.name = name
        self.klasse = klasse
        self.waffen = waffen_liste
    def __repr__(self):
        return f"{self.klasse} '{self.name}' mit Waffen: {self.waffen}"

# Ein spezifischer Bauplan für eine Fregatte
fregatten_bauplan = Schiffsklasse(
    "USS Reliant", 
    "Fregatte", 
    ["Phaser Typ-8", "Photonentorpedo Mk IV"]
)

# Speichern des Bauplans
with open("fregatte_bauplan.ship", "wb") as f:
    pickle.dump(fregatten_bauplan, f)
    print("Bauplan 'fregatte_bauplan.ship' wurde gespeichert.")
    
# Laden des Bauplans
with open("fregatte_bauplan.ship", "rb") as f:
    geladener_bauplan = pickle.load(f)
    print(f"Geladener Bauplan: {geladener_bauplan}")



Bauplan 'fregatte_bauplan.ship' wurde gespeichert.
Geladener Bauplan: Fregatte 'USS Reliant' mit Waffen: ['Phaser Typ-8', 'Photonentorpedo Mk IV']



**Anwendung von `shelve`:** Wir erstellen eine "Hangar"-Datenbank, um mehrere verschiedene Schiffe unter ihrer Registrierungsnummer zu verwalten.


In [9]:


import shelve

# Verschiedene Schiffe erstellen
enterprise = Schiffsklasse("Enterprise-D", "Galaxy", ["Phaser Typ-10", "Quantentorpedo"])
defiant = Schiffsklasse("Defiant", "Eskorte", ["Puls-Phaser", "Quantentorpedo"])

# Hangar-Datenbank öffnen und Schiffe "andocken"
with shelve.open("hangar_db") as hangar:
    hangar["NCC-1701-D"] = enterprise
    hangar["NX-74205"] = defiant
    
# Ein bestimmtes Schiff aus dem Hangar abrufen
with shelve.open("hangar_db") as hangar:
    abgerufenes_schiff = hangar["NX-74205"]
    print(f"\nSchiff aus Hangar abgerufen: {abgerufenes_schiff}")




Schiff aus Hangar abgerufen: Eskorte 'Defiant' mit Waffen: ['Puls-Phaser', 'Quantentorpedo']


## **5.3: Fortgeschrittene Ausnahmebehandlung**

Bisher haben wir Exceptions meist nur mit einem einfachen `try...except`-Block behandelt. In professionellen Anwendungen benötigen wir jedoch mehr Kontrolle und Spezifität, um robuste und gut wartbare Programme zu schreiben. In diesem Abschnitt lernen wir, wie wir unser Fehler-Handling auf das nächste Level heben.

---
---

### **5.3.1 Theorie & Beispiele: Eigene Exceptions definieren**

**Tiefe Erklärung: Warum `ValueError` nicht immer ausreicht**

Bisher haben wir bei Fehlern oft einen generischen `ValueError` oder `TypeError` ausgelöst. Das ist besser als nichts, aber in komplexen Systemen oft nicht aussagekräftig genug.

* **Problem 1: Mangelnde Spezifität:** Wenn eine Funktion einen `ValueError` auslösen kann, weiß der aufrufende Code nicht, *warum*. War die Eingabe zu niedrig? Zu hoch? Im falschen Format? Ein `MotorUeberhitztError` oder `UngueltigerBenutzernameError` ist unmissverständlich.
* **Problem 2: Unpräzises Error-Handling:** Der aufrufende Code kann nur `except ValueError:` schreiben. Damit fängt er aber vielleicht auch andere `ValueError`-Fälle ab, die er gar nicht behandeln wollte. Mit einer eigenen Exception kann man einen `except MotorUeberhitztError:`-Block schreiben, der *nur* auf dieses eine, spezifische Problem reagiert.
* **Problem 3: Fehlende API-Dokumentation:** Eigene Exceptions sind Teil der öffentlichen Schnittstelle (API) Ihres Codes. Sie dokumentieren klar und deutlich, welche spezifischen Fehler bei der Benutzung Ihrer Klasse oder Funktion auftreten können.

**Die Lösung: Eigene Exception-Klassen**

Die gute Nachricht ist: Das Erstellen eigener Exceptions ist extrem einfach. Man erstellt einfach eine neue Klasse, die von der eingebauten `Exception`-Klasse (oder einer spezifischeren Exception wie `ValueError`) erbt.


In [10]:


# Die einfachste mögliche eigene Exception
class MeinFehler(Exception):
    pass




Man kann diese Fehlerklassen auch mit eigenen `__init__`-Methoden erweitern, um zusätzliche Informationen zum Fehlerkontext zu speichern.

---

### **Beispiel: Die `GuthabenNichtAusreichendError`-Exception**

Wir bauen eine `Konto`-Klasse, die einen spezifischen Fehler auslöst, wenn eine Abhebung fehlschlägt.


In [11]:


# Schritt 1: Die eigene Exception definieren.
# Sie erbt von Exception, damit sie sich wie ein normaler Fehler verhält.
class GuthabenNichtAusreichendError(Exception):
    """Wird ausgelöst, wenn eine Abhebung das Guthaben übersteigen würde."""
    pass

class Konto:
    def __init__(self, inhaber, kontostand=0):
        self.inhaber = inhaber
        self.kontostand = kontostand
        
    def abheben(self, betrag):
        print(f"Versuche, {betrag} EUR abzuheben...")
        if betrag > self.kontostand:
            # Wir lösen unsere eigene, spezifische Exception aus.
            raise GuthabenNichtAusreichendError(
                f"Abhebung fehlgeschlagen. Guthaben: {self.kontostand}, benötigter Betrag: {betrag}"
            )
        else:
            self.kontostand -= betrag
            print("Abhebung erfolgreich.")
            
    def __str__(self):
        return f"Konto von {self.inhaber} | Stand: {self.kontostand} EUR"

# Schritt 2: Die Funktionalität in einem try...except-Block nutzen.
mein_konto = Konto("Anna", 100)
print(mein_konto)

try:
    mein_konto.abheben(50)  # Das funktioniert
    print(mein_konto)
    
    mein_konto.abheben(80)  # Das wird fehlschlagen
    print(mein_konto)
    
except GuthabenNichtAusreichendError as e:
    # Wir können jetzt gezielt auf DIESEN einen Fehler reagieren.
    print(f"\nFEHLER-HANDLING: Spezifischer Fehler aufgetreten!")
    print(f"Nachricht: {e}")
    print(f"Aktion: Sende Benachrichtigung an den Kunden...")
finally:
    print(f"\nFinaler Kontostand: {mein_konto}")



Konto von Anna | Stand: 100 EUR
Versuche, 50 EUR abzuheben...
Abhebung erfolgreich.
Konto von Anna | Stand: 50 EUR
Versuche, 80 EUR abzuheben...

FEHLER-HANDLING: Spezifischer Fehler aufgetreten!
Nachricht: Abhebung fehlgeschlagen. Guthaben: 50, benötigter Betrag: 80
Aktion: Sende Benachrichtigung an den Kunden...

Finaler Kontostand: Konto von Anna | Stand: 50 EUR



---
---

### **5.3.2 Exceptions verketten (`raise from`)**

**Erklärung: Den ursprünglichen Fehlerkontext nicht verlieren**

Oft passiert in unserem Code ein technischer Low-Level-Fehler (z.B. ein `FileNotFoundError`, `KeyError` oder `ConnectionError`), der für den Endnutzer unserer Funktion aber nicht aussagekräftig ist. Wir wollen diesen technischen Fehler fangen und eine eigene, aussagekräftigere High-Level-Exception auslösen.

**Die naive (und schlechte) Lösung:**


In [12]:


try:
    # ... code der einen KeyError auslöst ...
except KeyError:
    raise MeineEigeneException("Etwas ist schiefgelaufen")



IndentationError: expected an indented block after 'try' statement on line 1 (1447382666.py, line 3)


Das Problem hierbei: Der komplette Traceback des ursprünglichen `KeyError` geht verloren! Man sieht nur noch die `MeineEigeneException` und verliert wertvolle Informationen für das Debugging.

**Die professionelle Lösung: Explizite Verkettung mit `raise from`**

Python 3 bietet eine elegante Lösung, um beide Informationen zu erhalten: `raise ... from ...`.

* **Syntax:** `raise HighLevelException() from original_exception`
* **Mechanismus:** Diese Syntax tut zwei Dinge:
    1.  Sie löst die neue `HighLevelException` aus.
    2.  Sie speichert die `original_exception` im `__cause__`-Attribut der neuen Exception.
* **Ergebnis:** Der Python-Interpreter ist so klug, bei der Fehlerausgabe **beide** Tracebacks anzuzeigen. Zuerst den der ursprünglichen Ursache, und danach den der neuen, aussagekräftigeren Exception, die daraus resultierte. Man sieht die gesamte Fehlerkette.

---

### **Beispiel: Ein fehlertoleranter Konfigurations-Lader**

Wir schreiben eine Funktion, die eine Einstellung aus einer JSON-Datei liest. Sie soll aussagekräftige Fehler ausgeben, wenn die Datei nicht existiert oder der Schlüssel fehlt, aber trotzdem den Originalfehler für Entwickler beibehalten.


In [ ]:


import json

# Eigene High-Level-Exception
class KonfigurationsFehler(Exception):
    pass

def lade_einstellung(dateipfad, schluessel):
    print(f"\nLade Einstellung '{schluessel}' aus '{dateipfad}'...")
    try:
        with open(dateipfad, 'r') as f:
            daten = json.load(f)
        return daten[schluessel]
        
    except FileNotFoundError as e:
        # Fange den technischen Fehler und löse einen aussagekräftigeren aus,
        # behalte aber den Originalfehler als Ursache.
        raise KonfigurationsFehler(f"Die Konfigurationsdatei wurde nicht gefunden.") from e
        
    except KeyError as e:
        raise KonfigurationsFehler(f"Der Schlüssel '{schluessel}' fehlt in der Konfiguration.") from e
    
    except json.JSONDecodeError as e:
        raise KonfigurationsFehler("Die Konfigurationsdatei ist fehlerhaft (kein valides JSON).") from e

# Testfall 1: Datei nicht gefunden
try:
    lade_einstellung("nicht_existent.json", "user")
except KonfigurationsFehler as e:
    print(f"Abgefangen: {e}")
    # In einer echten Ausgabe würde man hier den kompletten Traceback beider Fehler sehen

# Erstellen einer gültigen Test-Datei
with open("config.json", "w") as f:
    f.write('{"host": "localhost", "port": 8080}')

# Testfall 2: Schlüssel nicht gefunden
try:
    lade_einstellung("config.json", "user")
except KonfigurationsFehler as e:
    print(f"Abgefangen: {e}")
    
# Testfall 3: Alles funktioniert
host = lade_einstellung("config.json", "host")
print(f"Erfolgreich geladen: host = {host}")




---
---

### **5.3.3 Transfer auf das `Raumschiff`-Projekt**

Wir definieren nun eigene Exceptions für unser Raumschiff und nutzen die Verkettung, um Fehler beim Zugriff auf Subsysteme klarer zu machen.


In [ ]:


# Schritt 1: Eigene, spezifische Exceptions definieren
class RaumschiffError(Exception):
    """Basis-Exception für alle Fehler, die mit dem Raumschiff zu tun haben."""
    pass

class SubsystemNichtGefundenError(RaumschiffError):
    """Wird ausgelöst, wenn ein Subsystem nicht existiert."""
    pass

class WarpkernInstabilError(RaumschiffError):
    """Wird ausgelöst, wenn der Warpkern kritisch ist."""
    pass

# Schritt 2: Die Raumschiff-Klasse, die diese Exceptions nutzt
class Raumschiff:
    def __init__(self, name):
        self.name = name
        # Subsysteme werden in einem Dictionary verwaltet
        self.subsysteme = {
            "Lebenserhaltung": {"status": "Online", "energie": 5},
            "Schilde": {"status": "Online", "energie": 10},
            "Warpkern": {"status": "Instabil", "energie": 0}
        }
        
    def energie_umleiten(self, von_system, zu_system, menge):
        print(f"\nLeite {menge} Energie von '{von_system}' zu '{zu_system}' um...")
        try:
            # Versuch, auf die Subsysteme zuzugreifen
            if self.subsysteme[von_system]['energie'] >= menge:
                self.subsysteme[von_system]['energie'] -= menge
                self.subsysteme[zu_system]['energie'] += menge
                print("Energie-Umleitung erfolgreich.")
            else:
                print("Nicht genug Energie im Quellsystem.")
                
        except KeyError as e:
            # Fange den technischen KeyError und löse unsere aussagekräftige Exception aus
            raise SubsystemNichtGefundenError(f"Das Subsystem '{e.args[0]}' ist nicht an Bord.") from e

# Testen
enterprise = Raumschiff("Enterprise")

try:
    # Dieser Aufruf funktioniert, da die Systeme existieren
    enterprise.energie_umleiten("Schilde", "Lebenserhaltung", 5)
    
    # Dieser Aufruf wird fehlschlagen, da "Phaser" nicht existiert
    enterprise.energie_umleiten("Schilde", "Phaser", 5)
    
except SubsystemNichtGefundenError as e:
    # Wir fangen unseren eigenen, sauberen Fehler ab
    print(f"\nFEHLER IM BORDCOMPUTER: {e}")
    # Der Traceback würde uns auch den ursprünglichen KeyError anzeigen
    if e.__cause__:
        print(f"Ursprünglicher technischer Fehler: {type(e.__cause__).__name__}")


# Kapitel 5.4: Tages-Challenge – Anwendung im `Smart Grid`-Projekt

**Heutiges Ziel:**

In dieser umfassenden Aufgabe werden wir alle heute gelernten Konzepte – **unabhängige Objektkopien, das Speichern und Laden von Projektzuständen und professionelles Fehler-Handling** – in unserem `Smart Grid`-Projekt anwenden. Wir werden eine Simulationsumgebung bauen, den Zustand unseres Netzwerks sichern und es mit spezifischen Fehlerklassen robuster machen.

---

## **Voraussetzungen: Benötigte Klassen**

Wir benötigen funktionierende Versionen unserer Klassen aus den vorherigen Kapiteln. Falls Ihr Code nicht zur Hand ist, können Sie diese vereinfachte Basis als Ausgangspunkt verwenden. Kopieren Sie sie in Ihre Zelle.



In [ ]:

import copy
import pickle

# --- Basisklassen aus vorherigen Kapiteln ---

class Energieerzeuger:
    """Stellt einen einzelnen Energieerzeuger im Netz dar."""
    NETZ_FREQUENZ = 50 # in Hz

    def __init__(self, name, max_leistung, aktuelle_leistung=0):
        self.name = name
        self.max_leistung = max_leistung
        self.aktuelle_leistung = aktuelle_leistung
        self.ist_aktiv = True

    def __repr__(self):
        return f"{self.name} ({self.aktuelle_leistung}/{self.max_leistung} MW)"

class Verteilerknoten:
    """Verwaltet eine Gruppe von Energieerzeugern."""
    def __init__(self, standort):
        self.standort = standort
        self._erzeuger = []

    def add_erzeuger(self, erzeuger):
        if isinstance(erzeuger, Energieerzeuger):
            self._erzeuger.append(erzeuger)
        else:
            raise TypeError("Nur Energieerzeuger-Objekte können hinzugefügt werden.")
    
    def __len__(self):
        return len(self._erzeuger)

    def __repr__(self):
        return f"Verteilerknoten '{self.standort}' mit {len(self)} Erzeugern."





---

### **Ihre Aufgaben**

#### **Aufgabe 1: Eigene Fehlerklassen definieren**

Definieren Sie zwei neue, spezifische Exception-Klassen, die von der eingebauten `Exception`-Klasse erben. Diese werden uns helfen, Fehler in unserem Netzwerk klar zu benennen.

1.  `UeberlastungsError(Exception)`: Soll ausgelöst werden, wenn die angeforderte Last die maximale Kapazität des Netzes übersteigt.
2.  `NetzwerkInstabilError(Exception)`: Soll ausgelöst werden, wenn eine andere kritische Bedingung eintritt.

```python
# Ihr Code hier für Aufgabe 1
```

---

#### **Aufgabe 2: Den `Verteilerknoten` mit Fehlerprüfung erweitern**

Erweitern Sie Ihre `Verteilerknoten`-Klasse um eine Methode zur Überprüfung der Netzlast.

1.  Fügen Sie eine Methode `gesamte_leistung_abrufen()` hinzu. Sie soll ein Tupel zurückgeben, das die Summe der `aktuellen_leistung` und die Summe der `max_leistung` aller angeschlossenen Erzeuger enthält.
2.  Fügen Sie eine zweite Methode `last_pruefen()` hinzu. Diese Methode soll `gesamte_leistung_abrufen()` aufrufen. Wenn die aktuelle Gesamtleistung größer als 80% der maximalen Gesamtkapazität ist, soll sie eine `UeberlastungsError` mit einer aussagekräftigen Fehlermeldung auslösen. Ansonsten soll sie eine Erfolgsmeldung ausgeben.

```python
# Erweitern Sie hier Ihre Verteilerknoten-Klasse
```

---

#### **Aufgabe 3: Netzwerkzustand speichern und laden (`pickle`)**

Machen Sie den Zustand Ihres `Verteilerknoten`-Objekts persistent.

1.  Implementieren Sie eine Methode `speichern(self, dateipfad)`. Diese Methode soll `pickle.dump()` verwenden, um die **gesamte `Verteilerknoten`-Instanz (`self`)** in die angegebene Datei zu schreiben.
2.  Implementieren Sie eine **`@classmethod`** `laden(cls, dateipfad)`. Diese Methode soll eine `Verteilerknoten`-Instanz aus einer Datei mit `pickle.load()` laden und das wiederhergestellte Objekt zurückgeben. Klassenmethoden sind hierfür gut geeignet, da wir ein neues Objekt erzeugen, ohne eine existierende Instanz zu benötigen.

```python
# Erweitern Sie hier Ihre Verteilerknoten-Klasse
```
---

#### **Aufgabe 4: Die `NetzwerkSimulation`-Klasse erstellen (`deepcopy`)**

Erstellen Sie eine neue Klasse `NetzwerkSimulation`, um Lastspitzen zu simulieren, ohne das echte Netzwerk zu beeinflussen.

1.  Der `__init__`-Konstruktor soll ein `Verteilerknoten`-Objekt (`original_netzwerk`) entgegennehmen.
2.  Innerhalb des Konstruktors muss eine **vollständig unabhängige Kopie** dieses Objekts mit `copy.deepcopy()` erstellt und als `self.simulations_netzwerk` gespeichert werden.
3.  Fügen Sie eine Methode `simuliere_lastspitze(self, erhoe hung_pro_erzeuger)` hinzu. Diese Methode soll:
    * Über alle Erzeuger im **simulierten** Netzwerk (`self.simulations_netzwerk._erzeuger`) iterieren und deren `aktuelle_leistung` um den Wert `erhoehung_pro_erzeuger` erhöhen.
    * Anschließend die `last_pruefen()`-Methode des simulierten Netzwerks in einem `try...except`-Block aufrufen, um eine mögliche `UeberlastungsError` abzufangen und eine entsprechende Nachricht auszugeben.

```python
# Ihr Code hier für Aufgabe 4
```

---
#### **Aufgabe 5: Alles zusammenfügen und testen**

Schreiben Sie nun den finalen Test-Code, der alle neuen Features demonstriert.

1.  **Original-Netzwerk aufbauen:** Erstellen Sie mehrere `Energieerzeuger`-Instanzen und fügen Sie sie einem `Verteilerknoten`-Objekt namens `hauptnetz` hinzu.
2.  **Zustand speichern:** Rufen Sie `hauptnetz.speichern("netz_backup.pkl")` auf.
3.  **Simulation durchführen:**
    * Erstellen Sie eine Instanz von `NetzwerkSimulation` und übergeben Sie ihr `hauptnetz`.
    * Rufen Sie `simuliere_lastspitze()` mit einem Wert auf, der eine `UeberlastungsError` auslöst.
4.  **Unabhängigkeit beweisen:** Geben Sie den Zustand des originalen `hauptnetz` nach der Simulation aus. Die Werte der `aktuellen_leistung` dürfen sich **nicht** verändert haben.
5.  **Zustand laden:** Erstellen Sie eine neue Variable `geladenes_netz = Verteilerknoten.laden("netz_backup.pkl")`. Geben Sie das geladene Objekt aus und überprüfen Sie, ob es dem ursprünglichen `hauptnetz` entspricht.

```python
# Ihr Code hier für Aufgabe 5
```